In [18]:
a = 9567000

In [4]:
str(a)[-3:]

'000'

In [5]:
str(a)[:-3]

'831'

In [16]:
len(str(a))

7

In [19]:
if len(str(a)) > 6:
    print("Rp " + str(a)[0] + "." + str(a)[1:-3] + "." + str(a)[-3:])
else:
    print("Rp " + str(a)[:-3] + "." + str(a)[-3:])

Rp 9.567.000


In [ ]:
rsvm = 1
fd = 1
cd = 1
mm = 1
alc = 1
wifi = 1
rpt = 1
wnb = 1
rate = 5
rev = 100
pvlt = 1
cock = 1
bft = 1
brt = 1
bir = 1
ent = 1
nl = 1
tw = 1
qb = 1

In [ ]:
feature = pd.DataFrame({
            'reservasimejadirekomendasikan' : [rsvm],
            'Fine Dining' : [fd],
            'Casual Dining' : [cd],
            'makanmewah' : [mm],
            'alkoholtersedia' : [alc],
            'wifitersedia' : [wifi],
            'ruangpribaditersedia' : [rpt],
            'hanyawinedanbir' : [wnb],
            'Restaurant Rating' : [rate],
            'Restaurant Review' : [rev],
            'parkirvalettersedia' : [pvlt],
            'menyediakancocktail' : [cock],
            'buffet' : [bft],
            'Barat' : [brt],
            'hanyabir' : [bir],
            'liveentertainment' : [ent],
            'kehidupanmalam' : [nl],
            'bawapulangtersedia' : [tw],
            'Quick Bites' : [qb]
        })

In [ ]:
feature

In [ ]:
scaled = pd.DataFrame(scaler.transform(feature), columns=feature.columns)
hasil = model.predict(scaled)
hasil = int(round(hasil[0]*1000,-2))

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
from clean import data_final, dftop25, readdata, readdatarecsys
from prediction import prediction
from py_recommendersystem import getrecommendationall
from prediction2 import prediction2
from plots import trend, sugarscale, gravity, ibu, color
from werkzeug.utils import secure_filename
import os
# from plots import figuretop, brewmethod

app=Flask(__name__)

UPLOAD_FOLDER = 'D:\Personal Data\Data Science\Personal Project\data\data\static\images'
ALLOWED_EXTENSION = set(['png', 'jpeg', 'jpg', 'gif'])
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSION

@app.route('/')
def home():
    return render_template ('home.html')

@app.route('/data')
def data():
    data = readdata()
    return render_template ('data.html', data=data)

@app.route('/plots')
def plots():
    data = readdata()
    return render_template('plots.html', data=data)


@app.route('/classify', methods=['GET', 'POST'])
def classify():
    if request.method == 'POST':
        file = request.files['file']

        if 'file' not in request.files:
            flash('No file part')
            return render_template('imageclassify.html')
        
        if file.filename == '':
            flash('No selected file')
            return render_template('imageclassify.html')
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return 'file ' + filename + ' saved ' + ' <a href="/classify">BACK</a>'
 
    return render_template('imageclassify.html')



@app.route('/recomenderall',methods= ['GET','POST'])
def recommendall():
    if request.method == 'POST':
        data = request.form
        name = data['nama']
        hasil_recommend_all = getrecommendationall(name)
        return render_template('resultrecomall.html', data=hasil_recommend_all)
    return render_template('recommend-all.html')


@app.route('/prediction',methods= ['GET','POST'])
def prediction_html():
    if request.method == 'POST':
        data = request.form
        # data=data.to_dict()
        # data['Color']=float(data['Color'])
        # data['ABV']=float(data['ABV'])

        rsvm = int(data['reservasimejarekom'])
        fd = int(data['finedining'])
        cd = int(data['casualdining'])
        mm = int(data['makanmewah'])
        alc = int(data['alkoholtersedia'])
        wifi = int(data['wifitersedia'])
        rpt = int(data['ruangprivate'])
        wnb = int(data['hanyawinedanbir'])
        rate = float(data['rating'])
        rev = float(data['review'])
        pvlt = int(data['parkirvalettersedia'])
        cock = int(data['cocktailtersedia'])
        bft = int(data['buffettersedia'])
        brt = int(data['barat'])
        bir = int(data['hanyabir'])
        ent = int(data['liveent'])
        nl = int(data['nightlife'])
        tw = int(data['takeawaytersedia'])
        qb = int(data['quickbites'])

        feature = pd.DataFrame({
            'reservasimejadirekomendasikan' : [rsvm],
            'Fine Dining' : [fd],
            'Casual Dining' : [cd],
            'makanmewah' : [mm],
            'alkoholtersedia' : [alc],
            'wifitersedia' : [wifi],
            'ruangpribaditersedia' : [rpt],
            'hanyawinedanbir' : [wnb],
            'Restaurant Rating' : [rate],
            'Restaurant Review' : [rev],
            'parkirvalettersedia' : [pvlt],
            'menyediakancocktail' : [cock],
            'buffet' : [bft],
            'Barat' : [brt],
            'hanyabir' : [bir],
            'liveentertainment' : [ent],
            'kehidupanmalam' : [nl],
            'bawapulangtersedia' : [tw],
            'Quick Bites' : [qb]
        })

        hasil=prediction(feature)
        return render_template('result.html', hasil_prediksi=hasil)
    return render_template('prediction.html')


#TIDAK PERLU
@app.route('/prediction2',methods= ['GET','POST'])
def prediction2_html():
    if request.method == 'POST':
        data = request.form
        data=data.to_dict()
        data['OG']=float(data['OG'])
        data['FG']=float(data['FG'])
        data['Size(L)']=float(data['Size(L)'])
        data['BoilSize']=float(data['BoilSize'])
        hasil=prediction2(data)
        return render_template('result2.html', hasil_prediksi2=hasil)
    return render_template('prediction2.html')



if __name__ == '__main__':
    app.run(debug=True,port=2000)




In [ ]:
# return 'file ' + filename + ' saved ' + ' <a href="/classify">BACK</a>'

In [1]:
True

True

In [2]:
int(True)

1